In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

In [10]:
def score(nodes,solution,prime,offset=0,raw=False):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    out = distance(s,e)
    n = len(s)
    if raw:
        return out
    isten = (np.arange(n)+1+offset)%10==0
    notprime = ~np.isin(solution[:-1],prime)
    isextra = (isten&notprime)*0.1
    out += out*isextra
    return out

def score_(*args):
    return score(*args).sum()

def distance(coor1,coor2):
    return np.sqrt(((coor1-coor2)**2).sum(axis=-1))

class problem:
    def __init__(self, instance_path = './problem/cities.csv', base_path = './solution/base.csv'):
        self.nodes = pd.read_csv(instance_path).iloc[:,1:].values
        self.n = n = len(self.nodes)
        if base_path:
            self.base = pd.read_csv(base_path).values[:,0]
            
        prime_ = sieve.primerange(0,n)
        self.prime = list(prime_)
        prime_ = sieve.primerange(0,n)
        self.prime_set = set(prime_)
    
    def solve(self, *args, **kwargs,):
        solution = two_reverse.solve(self.nodes, self.base, self.prime, self.prime_set, *args, **kwargs)
        print(score_(self.nodes,solution,self.prime))

In [11]:
p = problem()

In [12]:
p.solve(120)

Start iteration 1
iter 1 done, total swap 0, saves 0.000, time 1.8
1516912.3751145517
####################################
zero
1516912.3751145517


In [3]:
class two_reverse:
    
    @classmethod
    def solve(cls, nodes, solution, prime, prime_set, run_through=None, max_iter=5, start_at=1):
        solution = solution.copy()
        start = time.time()
        
        i=1
        if run_through == None:
            run_through = len(nodes)-1
        
        asd = nodes[solution]
        df = pd.DataFrame(asd)
        df.to_csv('asd.csv')
        solution_nodes = pd.read_csv('asd.csv').iloc[:,1:].values
        
        cls.nodes = nodes
        cls.solution = solution
        cls.prime = prime
        cls.prime_set = prime_set
        cls.solution_nodes = solution_nodes
        
        while i <= max_iter:
            print('Start iteration %s'%i)
            
            iter_start = time.time()
            best = []
            did = 0

            reverse_dist, old_dist, roller = cls.calc_basis(start_at)

            for left in range(start_at, run_through):
                node_before_a, node_a = solution[left-1:left+1]
                
                middle, reverse_dist = cls.calc_middle(reverse_dist, left)
                to_d, from_a, roller =  cls.calc_connector(roller, left, node_before_a, node_a)
                place, saving, old_dist = cls.calc_save(from_a,to_d,middle,old_dist)

                if saving<0:
                    best.append(saving)
                    did += saving
                    target = place+1+left

                    #execute swap
                    cls.excute_swap(left,place)
                    print('a: %s | saving: %.3f | cumsave: %.3f'%(left, saving, did))

                    #reset infos
                    reverse_dist, old_dist, roller = cls.calc_basis(left+1)
                    
            print('iter %s done, total swap %s, saves %.3f, time %.1f'%(i,len(best),did,time.time()-iter_start))
            print(score(nodes,solution,prime).sum())
            print('####################################')
            i+=1
            if len(best) == 0:
                print('zero')
                break
        return solution
    
    @classmethod
    def calc_reverse(cls,left):
        r_solution = cls.solution[left:-1][::-1]
        reverse_dist = []
        for offset in range(10):
            reverse_dist.append(np.cumsum(score(cls.nodes,r_solution,cls.prime,offset)[::-1])[::-1])
        reverse_dist = np.array(reverse_dist)
        return reverse_dist
    
    @classmethod
    def calc_old_dist(cls, from_node):
        offset = from_node%10
        return np.cumsum(score(cls.nodes, cls.solution[from_node:],cls.prime, offset=offset))
    
    @classmethod
    def calc_middle(cls, reverse_dist, left):
        middle = np.zeros(shape=(reverse_dist.shape[1]))
        offset = left%10

        for o in range(10):
            order = (offset-o)%10
            using_reverse = reverse_dist[order]
            middle[o::10] = using_reverse[o::10]

        reverse_dist[:,:-1] -= reverse_dist[:,-1:]
        reverse_dist = reverse_dist[:,:-1]
        return middle[::-1], reverse_dist
    
    @classmethod
    def calc_connector(cls, roller, left, node_before_a, node_a):
        to_d = roller[:-1].copy()
        offset = left%10
        if node_before_a not in cls.prime_set and offset == 0:
            to_d *= 1.1

        roller = distance(cls.nodes[node_a], cls.solution_nodes[left+2:])
        if node_a in cls.prime_set:
            return to_d, roller, roller  
        else:
            start_leg = (10-left+2)%10
            from_a = roller.copy()
            from_a[start_leg::10] *= 1.1
            return to_d, from_a, roller
    
    @classmethod
    def calc_save(cls, from_a,to_d,middle,old_dist):
        save = from_a+to_d-old_dist[2:]+middle
        old_dist[1:] -= old_dist[0]
        old_dist = old_dist[1:]

        place = np.argmin(save)
        saving = save[place]
        return place, saving, old_dist
    
    @classmethod
    def calc_basis(cls, start_at):
        reverse_dist = cls.calc_reverse(start_at)
        old_dist = cls.calc_old_dist(from_node=start_at-1)
        roller = distance(cls.nodes[cls.solution[start_at-1]], cls.solution_nodes[start_at-1+2:])

        return reverse_dist, old_dist, roller
    
    @classmethod
    def excute_swap(cls,left,place):
        target = place+1+left
        cls.solution[left:target+1] = cls.solution[target:left-1:-1]
        cls.solution_nodes[left:target+1] = cls.solution_nodes[target:left-1:-1]


In [313]:
def calc_reverse(left, nodes, solution):
    r_solution = solution[left:-1][::-1]
    reverse_dist = []
    for offset in range(10):
        reverse_dist.append(np.cumsum(score(nodes,r_solution,offset)[::-1])[::-1])
    reverse_dist = np.array(reverse_dist)
    
    return reverse_dist

def calc_old_dist(nodes, solution, from_node):
    offset = from_node%10
    return np.cumsum(score(nodes,solution[from_node:],offset=offset))

def calc_middle(reverse_dist, left):
    middle = np.zeros(shape=(reverse_dist.shape[1]))
    offset = left%10
    
    for o in range(10):
        order = (offset-o)%10
        using_reverse = reverse_dist[order]
        middle[o::10] = using_reverse[o::10]
    
    reverse_dist[:,:-1] -= reverse_dist[:,-1:]
    reverse_dist = reverse_dist[:,:-1]
    return middle[::-1], reverse_dist

def calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set):
    to_d = roller[:-1].copy()
    offset = left%10
    if node_before_a not in prime_set and offset == 0:
        to_d *= 1.1
        
    roller = distance(nodes[node_a],solution_nodes[left+2:])
    if node_a in prime_set:
        return to_d, roller, roller  
    else:
        start_leg = (10-left+2)%10
        from_a = roller.copy()
        from_a[start_leg::10] *= 1.1
        return to_d, from_a, roller
    
def calc_save(from_a,to_d,middle,old_dist):
    save = from_a+to_d-old_dist[2:]+middle
    old_dist[1:] -= old_dist[0]
    old_dist = old_dist[1:]
    
    place = np.argmin(save)
    saving = save[place]
    return place, saving, old_dist

def calc_basis(start_at, nodes, solution, solution_nodes):
    reverse_dist = calc_reverse(start_at, nodes, solution)
    old_dist = calc_old_dist(nodes, solution,from_node=start_at-1)
    roller = distance(nodes[solution[start_at-1]],solution_nodes[start_at-1+2:])
    
    return reverse_dist, old_dist, roller

def excute_swap(left,place,solution,solution_nodes):
    target = place+1+left
    
    solution[left:target+1] = solution[target:left-1:-1]
    solution_nodes[left:target+1] = solution_nodes[target:left-1:-1]
    

In [314]:
solution = pd.read_csv('./solution/base.csv').values[:,0]

asd = nodes[solution]
df = pd.DataFrame(asd)
df.to_csv('asd.csv')
solution_nodes = pd.read_csv('asd.csv').iloc[:,1:].values

score(nodes,solution).sum()

1516912.3751145517

In [316]:
bests = []
dids = []

start = time.time()

i=1
iter_num = 5
start_at = 1
run_through = 2000

while i <= iter_num:
    iter_start = time.time()
    best = []
    did = 0
    
    reverse_dist, old_dist, roller = calc_basis(start_at, nodes, solution, solution_nodes)

    for left in range(start_at, run_through):
        node_before_a, node_a = solution[left-1:left+1]

        middle, reverse_dist = calc_middle(reverse_dist, left)

        to_d, from_a, roller =  calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set)

        place, saving, old_dist = calc_save(from_a,to_d,middle,old_dist)
        
        if saving<0:
            
            best.append(saving)
            did += saving
            target = place+1+left
            
            #execute swap
            excute_swap(left,place,solution,solution_nodes)
            print('a: %s | saving: %.3f | cumsave: %.3f'%(left, saving, did))
            
            #reset infos
            reverse_dist, old_dist, roller = calc_basis(left+1, nodes, solution, solution_nodes)

    #pd.DataFrame({'Path':solution}).to_csv('./solution/neos_iter_%s.csv'%(i+1),index=False)
    bests.append(best)
    dids.append(did)
    print('iter %s done, total swap %s, saves %.3f, time %.1f'%(i,len(best),did,time.time()-iter_start))
    print(score(nodes,solution).sum())
    print('####################################')
    i+=1
    if len(best) == 0:
        print('zero')
        break
print(time.time()-start)

a: 1041 | saving: -0.738 | cumsave: -0.738
a: 1778 | saving: -0.524 | cumsave: -1.262
iter 1 done, total swap 2, saves -1.262, time 20.8
1516911.1133771057
####################################


KeyboardInterrupt: 

# UTILITY

In [20]:
#write result
pd.DataFrame({'Path':solution}).to_csv(path+'solution/submit2.csv',index=False)

In [21]:
#make .tsp problem
fname = path + 'problem/santa.tsp'
with open(fname, 'w') as f:
    f.write('NAME : santa\n')
    f.write('COMMENT : santa\n')
    f.write('TYPE : TSP\n')
    f.write('DIMENSION : %d\n' % n)
    f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
    f.write('NODE_COORD_SECTION\n')
    for idx,node in enumerate(nodes):
        x,y=node*100000
        f.write('%s %s %s\n' % (idx+1,x,y))
    f.write('EOF\n')

In [24]:
#read neos html
fname = path + 'solution/neos2.html'
l = 1
sol = []
with open(fname, 'r') as f:
    while l< 207274:
        line = f.readline()
        if l>=9505:
            sol.append(int(line.split(' ')[0]))
        l += 1
sol.append(0)
solution = np.array(sol)

In [279]:
#2-opt manual check calcucation
a,d = 185337,186129 
a_pre,d_nex = a-1,d+1

node_a,node_d,node_a_pre,node_d_nex = solution[[a,d,a_pre,d_nex]]
old = score(solution)[a_pre:d_nex].sum()

to_d  = distance(nodes[node_a_pre],nodes[node_d])
from_a = distance(nodes[node_a],nodes[node_d_nex])
check = (d+1)%10 ==0 and node_a not in prime
print(check)
if check:
    from_a*=1.1

partial = solution[185337:186129+1][::-1]
total = score(partial,offset = a%10).sum()

True


# node swap

In [283]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(solution)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values
oldnext = pd.DataFrame({'d':dist}).d.rolling(3,center=True).sum().values
notprime = ~np.isin(solution[:-1], prime)
isten = (np.arange(n)+1)%10==0
isextra = isten&notprime
better = 0

time_a = time.time()

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

swap = {}
for left in range(1,n-1):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)
    
    FROM_FLAG = (left+1)%10 == 0
    TO_FLAG = left%10 == 0
    LEFT_FLAG = notprime[left]
    LEFT_pre_FLAG = notprime[left-1]
    LEFT_nex_FLAG = notprime[left+1]
    
    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] = oldnext[left]

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    pre_ = pre[1:-1]
    nex_ = nex[1:-1]
    
    if TO_FLAG & LEFT_pre_FLAG:
        pre_ = pre_ + pre_*0.1
    if FROM_FLAG:
        nex_ = nex_ + nex_*notprime[left:]*0.1
        
    cur_1 = cur_1 + cur_1*isextra[left-1:-1]*0.1
    
    if LEFT_FLAG:
        cur_2 = cur_2 + cur_2*isten[left:]*0.1
    
    if FROM_FLAG & LEFT_nex_FLAG:
        adj_dist *= 1.1
    new_cost = pre_ + nex_ + cur_1 + cur_2
    
    
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = old_cost - new_cost
    good = np.where(save>0)[0]
    
    if len(good)>0:
        good_save = save[good]
        swap[left] = [good+left,good_save]
    
    if left%10000 == 0:
        time_b = time.time()
        print(left,time_b-time_a)
        time_a = time_b
print(time.time()-start)
print(len(swap))
###############################################
bk = set()
candidate = []
for left in swap:
    right,improve = swap[left]
    for i in range(len(right)):
        candidate.append((improve[i],right[i],left))
        
candidate = sorted(candidate)[::-1]
l=[]
r=[]
for improve,left,right in candidate:
    fill = [left-1,left,left+1,right-1,right,right+1]
    fill = [not (x in bk) for x in fill]
    if all(fill):
        l.append(left)
        r.append(right)
        bk.add(left-1)
        bk.add(left)
        bk.add(left+1)
        bk.add(right-1)
        bk.add(right)
        bk.add(right+1)

buf = solution[l]
solution[l] = solution[r]
solution[r] = buf

score(solution).sum()

10000 127.05045962333679
20000 114.68600010871887
30000 109.48599982261658
40000 103.19978618621826
50000 94.87000012397766
60000 89.02499985694885
70000 75.71099996566772
80000 52.21399998664856
90000 47.068000078201294
100000 42.63800001144409
110000 38.1710000038147
120000 33.9300000667572
130000 30.115999937057495
140000 23.044999837875366
150000 18.130000114440918
160000 15.259000062942505
170000 10.917999982833862
180000 7.343999862670898
190000 4.000999927520752
1038.1812453269958
0


1516677.4252172827